In [1]:
import numpy as np
import pandas as pd
import os
import glob
import superskrypt as sp
from scipy import stats

In [2]:
PTH = r"C:\Users\LEON-NETB\Dropbox\LEON_CIECHANOWSKI\DANE_EEG"
os.chdir(PTH)
print(os.getcwd())

C:\Users\LEON-NETB\Dropbox\LEON_CIECHANOWSKI\DANE_EEG


In [3]:
def is_odd(num):
    return num % 2

In [4]:
# counts SoA and RT for all subjects and concatenates them in a DataFrame 

db_ALL = pd.DataFrame()
db_RT = pd.DataFrame()
subj_num = []
for f in glob.glob('*.txt'):
    subj_num.append(int(f.split('.')[0][1:]))
    df = pd.read_csv(f, sep='\t')
    db_SoA = sp.get_color_info(df)
    db_RT_temp = sp.get_RT(df)
    db_ALL = db_ALL.append(db_SoA, ignore_index=True)
    db_RT = db_RT.append(db_RT_temp, ignore_index=True)
    print('read {}!'.format(f))
db_ALL.index = subj_num
db_RT.index = subj_num
print('Done!')

read s2.txt!
read s3.txt!
Done!


D:\Miniconda3\envs\py3\lib\site-packages\pandas\core\indexing.py:415: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [5]:
for col in db_RT.columns:
    for i in db_ALL.index:
        db_ALL.loc[i, '{}'.format(col)] = db_RT.loc[i, col]
    print('Done col: {}'.format(col))

Done col: comp_r_RT
Done col: comp_l_RT
Done col: incomp_r_RT
Done col: incomp_l_RT
Done col: neut_r_RT
Done col: neut_l_RT
Done col: comp_RT
Done col: incomp_RT
Done col: neut_RT


In [6]:
# writing means of each condition (comp_l... etc):

for c in db_ALL.columns:
    for i in db_ALL.index:
        db_ALL.loc[i, c] = np.mean(db_ALL.loc[i, c])

In [7]:
# means of conditions of both hands:

for c in db_ALL.columns:
    for i in db_ALL.index:
        db_ALL.loc[i, 'comp'] = (db_ALL.loc[i, 'comp_l']+db_ALL.loc[i, 'comp_r'])/2
        db_ALL.loc[i, 'incomp'] = (db_ALL.loc[i, 'incomp_l']+db_ALL.loc[i, 'incomp_r'])/2
        db_ALL.loc[i, 'neut'] = (db_ALL.loc[i, 'neut_l']+db_ALL.loc[i, 'neut_r'])/2

In [8]:
# adding info about the condition

for i in db_ALL.index:
    if is_odd(i):
        db_ALL.loc[i, 'condition'] = 'info'
    else:
        db_ALL.loc[i, 'condition'] = 'noinfo'

In [9]:
db_ALL = db_ALL.loc[np.sort(db_ALL.index), :]
db_ALL.to_excel('BAZA_NONSTAND.xlsx')